In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
964893,2021-02-03,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
964894,2021-02-04,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
964895,2021-02-05,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
964896,2021-02-06,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
26331,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
26333,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
26335,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
26337,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
26339,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
599019,2021-02-03,Arkansas,Arkansas,5001,1818,36.00,5000,Arkansas,AR,Arkansas,State,3017804
599021,2021-02-04,Arkansas,Arkansas,5001,1837,36.00,5000,Arkansas,AR,Arkansas,State,3017804
599023,2021-02-05,Arkansas,Arkansas,5001,1855,36.00,5000,Arkansas,AR,Arkansas,State,3017804
599025,2021-02-06,Arkansas,Arkansas,5001,1870,37.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
964893,2021-02-03,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
964894,2021-02-04,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
964895,2021-02-05,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
964896,2021-02-06,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-02-07') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
383,2021-02-07,Snohomish,Washington,29177,494.00,53061,WA,County,822083,3549.16,60.09
764,2021-02-07,Cook,Illinois,460554,9561.00,17031,IL,County,5150233,8942.39,185.64
1144,2021-02-07,Orange,California,252311,3358.00,6059,CA,County,3175692,7945.07,105.74
1523,2021-02-07,Maricopa,Arizona,487042,7971.00,4013,AZ,County,4485414,10858.35,177.71
1902,2021-02-07,Los Angeles,California,1146951,18044.00,6037,CA,County,10039107,11424.83,179.74
...,...,...,...,...,...,...,...,...,...,...,...
964549,2021-02-07,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08
964667,2021-02-07,King,Texas,13,0.00,48269,TX,County,272,4779.41,0.00
964754,2021-02-07,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00
964837,2021-02-07,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
383,2021-02-07,Snohomish,Washington,29177,494.00,53061,WA,County,822083,3549.16,60.09,60.09,3549.16
764,2021-02-07,Cook,Illinois,460554,9561.00,17031,IL,County,5150233,8942.39,185.64,185.64,8942.39
1144,2021-02-07,Orange,California,252311,3358.00,6059,CA,County,3175692,7945.07,105.74,105.74,7945.07
1523,2021-02-07,Maricopa,Arizona,487042,7971.00,4013,AZ,County,4485414,10858.35,177.71,177.71,10858.35
1902,2021-02-07,Los Angeles,California,1146951,18044.00,6037,CA,County,10039107,11424.83,179.74,179.74,11424.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...
964549,2021-02-07,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08,75.08,1651.65
964667,2021-02-07,King,Texas,13,0.00,48269,TX,County,272,4779.41,0.00,0.00,4779.41
964754,2021-02-07,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00,0.00,4238.26
964837,2021-02-07,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00,0.00,591.72


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
681245,2021-02-07,Clarke,Alabama,3331,40.00,1025,AL,County,23622,14101.26,169.33,169.33,14101.26,1
626200,2021-02-07,Hale,Alabama,2001,56.00,1065,AL,County,14651,13657.77,382.23,382.23,13657.77,2
490581,2021-02-07,Lowndes,Alabama,1269,42.00,1085,AL,County,9726,13047.50,431.83,431.83,13047.50,3
435227,2021-02-07,Etowah,Alabama,12800,273.00,1055,AL,County,102268,12516.13,266.95,266.95,12516.13,4
365496,2021-02-07,Franklin,Alabama,3910,68.00,1059,AL,County,31362,12467.32,216.82,216.82,12467.32,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
918186,2021-02-07,Platte,Wyoming,560,11.00,56031,WY,County,8393,6672.23,131.06,131.06,6672.23,19
597762,2021-02-07,Converse,Wyoming,921,14.00,56009,WY,County,13822,6663.29,101.29,101.29,6663.29,20
825063,2021-02-07,Niobrara,Wyoming,147,2.00,56027,WY,County,2356,6239.39,84.89,84.89,6239.39,21
742087,2021-02-07,Lincoln,Wyoming,1161,11.00,56023,WY,County,19830,5854.77,55.47,55.47,5854.77,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
490581,2021-02-07,Lowndes,Alabama,1269,42.00,1085,AL,County,9726,13047.50,431.83,431.83,13047.50,3,1
433947,2021-02-07,Clay,Alabama,1360,52.00,1027,AL,County,13235,10275.78,392.90,392.90,10275.78,24,2
626200,2021-02-07,Hale,Alabama,2001,56.00,1065,AL,County,14651,13657.77,382.23,382.23,13657.77,2,3
528858,2021-02-07,Greene,Alabama,825,29.00,1063,AL,County,8111,10171.37,357.54,357.54,10171.37,26,4
489943,2021-02-07,Crenshaw,Alabama,1353,49.00,1041,AL,County,13772,9824.28,355.79,355.79,9824.28,32,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527585,2021-02-07,Hot Springs,Wyoming,357,3.00,56017,WY,County,4413,8089.73,67.98,67.98,8089.73,15,19
723738,2021-02-07,Uinta,Wyoming,1983,12.00,56041,WY,County,20226,9804.21,59.33,59.33,9804.21,6,20
742087,2021-02-07,Lincoln,Wyoming,1161,11.00,56023,WY,County,19830,5854.77,55.47,55.47,5854.77,22,21
208394,2021-02-07,Teton,Wyoming,3238,7.00,56039,WY,County,23464,13799.86,29.83,29.83,13799.86,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,60.09,3549.16,15,21
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,60.09,3549.16,15,21
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,60.09,3549.16,15,21
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,60.09,3549.16,15,21
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,60.09,3549.16,15,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
962611,2021-02-03,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
962612,2021-02-04,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
962613,2021-02-05,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
962614,2021-02-06,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
678651,2020-04-01,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,169.33,14101.26,50,1,2.00
678652,2020-04-02,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,169.33,14101.26,50,1,0.00
678653,2020-04-03,Clarke,Alabama,3,0.00,1025,AL,County,23622,12.70,0.00,169.33,14101.26,50,1,1.00
678654,2020-04-04,Clarke,Alabama,7,0.00,1025,AL,County,23622,29.63,0.00,169.33,14101.26,50,1,4.00
678655,2020-04-05,Clarke,Alabama,9,0.00,1025,AL,County,23622,38.10,0.00,169.33,14101.26,50,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
754371,2021-02-03,Crook,Wyoming,415,10.00,56011,WY,County,7584,5472.05,131.86,131.86,5498.42,7,23,1.00
754372,2021-02-04,Crook,Wyoming,416,10.00,56011,WY,County,7584,5485.23,131.86,131.86,5498.42,7,23,1.00
754373,2021-02-05,Crook,Wyoming,417,10.00,56011,WY,County,7584,5498.42,131.86,131.86,5498.42,7,23,1.00
754374,2021-02-06,Crook,Wyoming,417,10.00,56011,WY,County,7584,5498.42,131.86,131.86,5498.42,7,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
488615,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,431.83,13047.50,1,3,1.00,0.00
488616,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,431.83,13047.50,1,3,0.00,0.00
488617,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,431.83,13047.50,1,3,0.00,0.00
488618,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,431.83,13047.50,1,3,0.00,0.00
488619,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,431.83,13047.50,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
487015,2021-02-03,Albany,Wyoming,3791,11.00,56001,WY,County,38880,9750.51,28.29,28.29,9837.96,23,5,1.00,0.00
487016,2021-02-04,Albany,Wyoming,3815,11.00,56001,WY,County,38880,9812.24,28.29,28.29,9837.96,23,5,24.00,0.00
487017,2021-02-05,Albany,Wyoming,3818,11.00,56001,WY,County,38880,9819.96,28.29,28.29,9837.96,23,5,3.00,0.00
487018,2021-02-06,Albany,Wyoming,3823,11.00,56001,WY,County,38880,9832.82,28.29,28.29,9837.96,23,5,5.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-02-07') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
249764,2021-02-07,Imperial,California,26514,559.00,6025,CA,County,181215,14631.24,308.47,308.47,14631.24,1,2,40.00,0.00
1902,2021-02-07,Los Angeles,California,1146951,18044.00,6037,CA,County,10039107,11424.83,179.74,179.74,11424.83,2,5,3034.00,89.00
436459,2021-02-07,Inyo,California,1113,29.00,6027,CA,County,18039,6169.97,160.76,160.76,6169.97,3,32,9.00,0.00
55913,2021-02-07,Stanislaus,California,52806,861.00,6099,CA,County,550660,9589.58,156.36,156.36,9589.58,4,10,244.00,0.00
69270,2021-02-07,Tulare,California,45349,645.00,6107,CA,County,466195,9727.47,138.35,138.35,9727.47,5,9,211.00,0.00
25992,2021-02-07,Riverside,California,280170,3320.00,6065,CA,County,2470546,11340.41,134.38,134.38,11340.41,6,6,0.00,0.00
395709,2021-02-07,Merced,California,27134,362.00,6047,CA,County,277680,9771.68,130.37,130.37,9771.68,7,8,0.00,0.00
528800,2021-02-07,Kings,California,21224,194.00,6031,CA,County,152940,13877.34,126.85,126.85,13877.34,8,3,5.00,2.00
25316,2021-02-07,Fresno,California,90944,1224.00,6019,CA,County,999101,9102.58,122.51,122.51,9102.58,9,14,431.00,0.00
44525,2021-02-07,San Joaquin,California,63619,893.00,6077,CA,County,762148,8347.33,117.17,117.17,8347.33,10,18,0.00,0.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
917737,2021-02-07,Lassen,California,5435,18.00,6035,CA,County,30573,17777.12,58.88,58.88,17777.12,34,1,0.00,0.00
249764,2021-02-07,Imperial,California,26514,559.00,6025,CA,County,181215,14631.24,308.47,308.47,14631.24,1,2,40.00,0.00
528800,2021-02-07,Kings,California,21224,194.00,6031,CA,County,152940,13877.34,126.85,126.85,13877.34,8,3,5.00,2.00
124690,2021-02-07,San Bernardino,California,277949,2125.00,6071,CA,County,2180085,12749.46,97.47,97.47,12749.46,13,4,0.00,0.00
1902,2021-02-07,Los Angeles,California,1146951,18044.00,6037,CA,County,10039107,11424.83,179.74,179.74,11424.83,2,5,3034.00,89.00
25992,2021-02-07,Riverside,California,280170,3320.00,6065,CA,County,2470546,11340.41,134.38,134.38,11340.41,6,6,0.00,0.00
156645,2021-02-07,Kern,California,97249,659.00,6029,CA,County,900202,10803.02,73.21,73.21,10803.02,26,7,387.00,0.00
395709,2021-02-07,Merced,California,27134,362.00,6047,CA,County,277680,9771.68,130.37,130.37,9771.68,7,8,0.00,0.00
69270,2021-02-07,Tulare,California,45349,645.00,6107,CA,County,466195,9727.47,138.35,138.35,9727.47,5,9,211.00,0.00
55913,2021-02-07,Stanislaus,California,52806,861.00,6099,CA,County,550660,9589.58,156.36,156.36,9589.58,4,10,244.00,0.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
249764,2021-02-07,Imperial,California,26514,559.00,6025,CA,County,181215,14631.24,308.47,308.47,14631.24,1,2,40.00,0.00
1902,2021-02-07,Los Angeles,California,1146951,18044.00,6037,CA,County,10039107,11424.83,179.74,179.74,11424.83,2,5,3034.00,89.00
436459,2021-02-07,Inyo,California,1113,29.00,6027,CA,County,18039,6169.97,160.76,160.76,6169.97,3,32,9.00,0.00
55913,2021-02-07,Stanislaus,California,52806,861.00,6099,CA,County,550660,9589.58,156.36,156.36,9589.58,4,10,244.00,0.00
69270,2021-02-07,Tulare,California,45349,645.00,6107,CA,County,466195,9727.47,138.35,138.35,9727.47,5,9,211.00,0.00
25992,2021-02-07,Riverside,California,280170,3320.00,6065,CA,County,2470546,11340.41,134.38,134.38,11340.41,6,6,0.00,0.00
395709,2021-02-07,Merced,California,27134,362.00,6047,CA,County,277680,9771.68,130.37,130.37,9771.68,7,8,0.00,0.00
528800,2021-02-07,Kings,California,21224,194.00,6031,CA,County,152940,13877.34,126.85,126.85,13877.34,8,3,5.00,2.00
25316,2021-02-07,Fresno,California,90944,1224.00,6019,CA,County,999101,9102.58,122.51,122.51,9102.58,9,14,431.00,0.00
44525,2021-02-07,San Joaquin,California,63619,893.00,6077,CA,County,762148,8347.33,117.17,117.17,8347.33,10,18,0.00,0.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,308.47,14631.24,1,2,4.00,0.00
1,03/21/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,308.47,14631.24,1,2,0.00,0.00
2,03/22/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,308.47,14631.24,1,2,0.00,0.00
3,03/23/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,308.47,14631.24,1,2,0.00,0.00
4,03/24/20,Imperial,California,9,0.00,6025,CA,County,181215,4.97,0.00,308.47,14631.24,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3262,02/03/21,Lassen,California,5421,17.00,6035,CA,County,30573,17731.33,55.60,58.88,17777.12,34,1,8.00,1.00
3263,02/04/21,Lassen,California,5435,17.00,6035,CA,County,30573,17777.12,55.60,58.88,17777.12,34,1,14.00,0.00
3264,02/05/21,Lassen,California,5435,17.00,6035,CA,County,30573,17777.12,55.60,58.88,17777.12,34,1,0.00,0.00
3265,02/06/21,Lassen,California,5435,18.00,6035,CA,County,30573,17777.12,58.88,58.88,17777.12,34,1,0.00,1.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,308.47,14631.24,1,2,4.00,0.00
1,03/21/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,308.47,14631.24,1,2,0.00,0.00
2,03/22/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,308.47,14631.24,1,2,0.00,0.00
3,03/23/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,308.47,14631.24,1,2,0.00,0.00
4,03/24/20,Imperial,California,9,0.00,6025,CA,County,181215,4.97,0.00,308.47,14631.24,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3336,02/03/21,San Joaquin,California,62890,874.00,6077,CA,County,762148,8251.68,114.68,117.17,8347.33,10,18,153.00,3.00
3337,02/04/21,San Joaquin,California,63258,878.00,6077,CA,County,762148,8299.96,115.20,117.17,8347.33,10,18,368.00,4.00
3338,02/05/21,San Joaquin,California,63619,893.00,6077,CA,County,762148,8347.33,117.17,117.17,8347.33,10,18,361.00,15.00
3339,02/06/21,San Joaquin,California,63619,893.00,6077,CA,County,762148,8347.33,117.17,117.17,8347.33,10,18,0.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)